# Exercise 5: Executing a Tool

In the previous exercise, the model asked us to call a tool. Now we'll complete the round-trip: execute the tool and send the result back.

By the end, you'll understand:
- How to append the assistant's tool-call response to the message list
- How to construct a `tool` role message with the result
- The full request/response cycle for tool use

## Step 1: Setup (same as previous exercise)

In [ ]:
import json
import openai

client = openai.OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        },
    }
]

## Step 2: Get the tool-call response

In [ ]:
messages = [{"role": "user", "content": "What is the weather in Tokyo?"}]

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages,
    tools=tools,
)

message = response.choices[0].message
print(f"finish_reason: {response.choices[0].finish_reason}")
print(f"tool_calls: {message.tool_calls}")

## Step 3: Append the assistant's response

**Important:** You must append the full message object (not just the text). This preserves the `tool_calls` field, which the API needs to match tool results back to their requests.

In [ ]:
messages.append(message)

## Step 4: Find the tool call and "execute" the tool

In a real application, you'd call an actual weather API here. For now, we return a hardcoded result.

Note: `function.arguments` is a **JSON string** — you need to parse it with `json.loads()`.

In [ ]:
tool_call = message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

print(f"The model wants to call: {tool_call.function.name}")
print(f"With arguments: {args}")
print(f"Tool call ID: {tool_call.id}")

# "Execute" the tool (hardcoded for now)
tool_result = "65 degrees"

## Step 5: Send the tool result back

We send the result as a message with `role: "tool"`. The `tool_call_id` must match the `id` from the tool call — this is how the model knows which call this result is for.

Each tool result is a **separate message** (unlike Anthropic where they're grouped in one user message).

In [ ]:
messages.append(
    {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": tool_result,
    }
)

final_response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages,
    tools=tools,
)

print(final_response.choices[0].message.content)

## Step 6: Review the full message flow

Let's look at the complete conversation to see the three-step pattern:
1. **User** asks a question
2. **Assistant** responds with `tool_calls`
3. **Tool** sends back the result
4. **Assistant** gives the final natural language answer

In [ ]:
messages.append(final_response.choices[0].message)

for msg in messages:
    if hasattr(msg, 'role'):
        role = msg.role
        content = msg.content or f"[tool_calls: {msg.tool_calls}]"
    else:
        role = msg['role']
        content = msg.get('content', '')
    print(f"--- {role} ---")
    print(f"{content}\n")

## Key takeaway

The tool-use cycle is:

**You → Model:** "What's the weather?" (+ tool definitions)  
**Model → You:** "Please call `get_weather` with `{"location": "Tokyo"}`"  
**You → Model:** "Here's the result: 65 degrees"  
**Model → You:** "The weather in Tokyo is 65 degrees."

In the next exercise, we'll automate this into a loop so it can handle multiple tool calls.